In [37]:
import requests
from bs4 import BeautifulSoup

# 29 April 2020 - present

this works for all* 2022 cases

*MOST -- some 2022 cases have the same problem as below (Akdogan, PWR, Bloomberg)

for 2020/2021 cases, the problem is just multiple p tags for the issue and fact h4 tags--need to address this so the dictionary doesn't get messed up. should work otherwise

In [108]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2021-0160.html')
soup = BeautifulSoup(response.content, "html.parser")

In [109]:
case_details = []
for detail in soup.find_all("div", class_="panel-filled"): 
    case_name = detail.find("span").string
    case_id = detail.find("h3", class_="sc-access").string
    case_id = case_id.replace('Case ID: ', '')
    keys = []
    for d in detail.find_all("h4"):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all("p"):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Case ID': case_id, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(len(case_details['Keys']))}
case_dict['Name'] = case_details['Name']
case_dict['Case ID'] = case_details['Case ID']
case_dict

{'Issue': 'Did the Supreme Court wrongly decide that Mr Crosland’s disclosure of the result of the Heathrow appeal, in breach of an embargo on the Court’s judgment, constituted a contempt of court? Did the Court then wrongly impose a fine of £5,000 on Mr Crosland, and wrongly order him to pay the Attorney General’s costs in the sum of £15,000?',
 'Facts': 'On 7 and 8 October 2020, the Supreme Court heard an appeal in the case of R (Friends of the Earth Ltd and others) v Heathrow Airport Limited [2020] UKSC 53 ("the Heathrow appeal"). Mr Timothy Crosland, an unregistered barrister, represented the charity Plan B Earth in those proceedings, in his capacity as a director of Plan B Earth.',
 'Appellant(s)': 'On 9 December 2020, a copy of the Supreme Court’s draft judgment was circulated to the parties’ representatives, to enable them to make suggestions for the correction of any errors, to prepare submissions on consequential matters, and to prepare themselves for the publication of the ju

# 13 March 2019 - April 1 2020

this works for ALL cases 13 March 2019 - April 1 2020


WORKS FOR "R v Chaytor and others (Appellants)" 01 DEC 2010
WORKS FOR "Greater Glasgow Health Board..." 17 DEC 2014

In [237]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2013-0124.html')
soup = BeautifulSoup(response.content, "html.parser")

In [238]:
case_details = []
for detail in soup.find_all("div", class_="panel-filled"):
    case_name = detail.find("span").string
    keys = []
    for d in detail.find_all("h3", class_="sc-access"):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all("p"):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
case_dict['Name'] = case_details['Name']
case_dict

{'Judgment date': '17 Dec 2014',
 'Neutral citation number': '[2014] UKSC 68',
 'Case ID': 'UKSC 2013/0124',
 'Justices': 'Lady Hale, Lord Wilson, Lord Reed, Lord Hughes, Lord Hodge',
 'Name': 'Greater Glasgow Health Board (Appellant) v Doogan and another (Respondents) (Scotland)'}

# 29 October 2009 - 6 March 2019

works from October 2009 (first case) - 6 March 2019

EXCEPT DEC 2014 -- Loveridge Appellant, Greater Glasgow  **New function for this one defined below**

EXCEPT DEC 2012 -- Spiller and other, R v Chaytor **New function for this one defined below**

EXCEPT cases with three citation numbers (first three Jan 2010) 

In [235]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0129.html')
soup = BeautifulSoup(response.content, "html.parser")

In [236]:
case_details = []
for detail in soup.find_all("div", class_="panel-filled"):
    case_name = detail.find("span").string
    keys = []
    for d in detail.find_all("h5"):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all("p"):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
case_dict['Name'] = case_details['Name']
case_dict

{'Judgment date': '10 Feb 2010',
 'Neutral citation number': '[2010] UKSC 6',
 'Case ID': 'UKSC 2009/0129',
 'Justices': 'Lord Hope, Lord Rodger, Lord Walker, Lord Brown, Lord Kerr',
 'Name': "Allison (Appellant) v Her Majesty's Advocate (Respondent) (Scotland) "}

# 3 December 2014

Loveridge (Appellant)

In [215]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2013-0273.html')
soup = BeautifulSoup(response.content, "html.parser")

In [216]:
case_details = []
for detail in soup.find_all("div", class_="panel-filled"):
    case_name = detail.find("span").string
    keys = ['Judgment date']
    for d in detail.find_all("h4"):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all("p"):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(len(case_details['Values']))}
case_dict['Name'] = case_details['Name']
case_dict

{'Judgment date': '03 Dec 2014',
 'Neutral citation number': '[2014] UKSC 65',
 'Case ID': 'UKSC 2013/0273',
 'Justices': 'Lord Neuberger, Lord Wilson, Lord Sumption, Lord Carnwath, Lord Toulson',
 'Name': 'Loveridge (Appellant) v Mayor and Burgesses of the London Borough of Lambeth (Respondent) '}

# 01 December 2010

Spiller and another (Appellant)

In [245]:
response = requests.get('https://www.supremecourt.uk/cases/uksc-2009-0210.html')
soup = BeautifulSoup(response.content, "html.parser")

In [247]:
case_details = []
for detail in soup.find_all("div", class_="panel-filled"):
    case_name = detail.find("h3").string
    keys = []
    for d in detail.find_all("h5"):
        key = d.string
        keys.append(key)
    values = []
    for d in detail.find_all("p"):
        value = d.string
        values.append(value)
    case_details.append({'Name': case_name, 'Keys': keys, 'Values': values})
    case_details = case_details[0]

case_dict = {case_details['Keys'][i]: case_details['Values'][i] for i in range(4)}
case_dict['Name'] = case_details['Name']
case_dict

{'Judgment date': '01 Dec 2010',
 'Neutral citation number': '[2010] UKSC 53',
 'Case ID': 'UKSC 2009/0210',
 'Justices': 'Lord Phillips, Lord Rodger, Lord Walker, Lord Brown, Sir John Dyson',
 'Name': 'Spiller and another (Appellants) v Joseph and others (Respondents)'}